In [ ]:
import sys
import os
# test 
# this is Cullan's code to run and plot the second gen mocks with all appropriate settings 

sys.path.append("..")
sys.path.append("../..")
sys.path.append("../../Barry/")
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
import numpy as np
import scipy as sp
import pandas as pd
from barry.models.model import Correction
from barry.utils import weighted_avg_and_cov
import matplotlib.colors as mplc
import matplotlib.pyplot as plt
import pickle
from chainconsumer import ChainConsumer, Chain, Truth, PlotConfig
colors = [mplc.cnames[color] for color in ["orange", "orangered", "firebrick", "lightskyblue", "steelblue", "seagreen", "black"]]

# read in Barry fitter object  

In [ ]:
t = 'LRGs'
z = [0.4, 0.6] 
mocks = 'LRGs_z04_06_pk'
main = '/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/desi_kp4_SecondGen_'
pfn = main + mocks + '-reducedcov/output/desi_kp4_SecondGen_' + mocks + '-reducedcov.fitter.pkl'

with open(pfn, 'rb') as pickle_file:
    fitter = pickle.load(pickle_file)

tracers = {
       "LRG": [[0.4, 0.6], [0.6, 0.8], [0.8, 1.1]],
       "ELG_LOPnotqso": [[0.8, 1.1], [1.1, 1.6]],
       "QSO": [[0.8, 2.1]],
       "BGS_BRIGHT-21.5": [[0.1, 0.4]],
        "ELGsLRGscombined": [[0.8, 1.1]]
    }
reconsmooth = {"LRG": 15, "ELG_LOPnotqso": 15, "QSO": 30, "BGS_BRIGHT-21.5": 15}
sigma_nl_par = {
    "LRG": [
        [9.0, 6.0],
        [9.0, 6.0],
        [9.0, 6.0],
    ],
    "ELG_LOPnotqso": [[8.5, 6.0], [8.5, 6.0]],
    "QSO": [[9.0, 6.0]],
    "BGS_BRIGHT-21.5": [[10.0, 8.0]],
    "ELGsLRGscombined": [[9.0, 6.0]]
}
sigma_nl_perp = {
    "LRG": [
        [4.5, 3.0],
        [4.5, 3.0],
        [4.5, 3.0],
    ],
    "ELG_LOPnotqso": [[4.5, 3.0], [4.5, 3.0]],
    "QSO": [[3.5, 3.0]],
    "BGS_BRIGHT-21.5": [[6.5, 3.0]],
    "ELGsLRGscombined": [[4.5, 3.0]]
    
}
sigma_s = {
    "LRG": [[2.0, 2.0], [2.0, 2.0], [2.0, 2.0]],
    "ELG_LOPnotqso": [[2.0, 2.0], [2.0, 2.0]],
    "QSO": [[2.0, 2.0]],
    "BGS_BRIGHT-21.5": [[2.0, 2.0]],
    "ELGsLRGscombined": [[2.0, 2.0]]
    
}

# function for plotting error histogram 

In [ ]:
def plot_errors(stats, figname, varybetaphaseshift=False, anisotropic=True):

    covs = np.cov(stats, rowvar=False)    
    nstats = len(stats)
    means = np.mean(stats, axis=0)

    labels, colors, fig, axes, zipp = None, None, None, None, None 
    
    if anisotropic: 
        labels = [r"$\sigma_{\alpha}$", r"$\sigma_{\alpha_{ap}}$", r"$\chi^{2}$"]
        colors = ["r", "b", "g"]
        fig, axes = plt.subplots(figsize=(7, 2), nrows=1, ncols=3, sharey=True, squeeze=False)
        plt.subplots_adjust(left=0.1, top=0.95, bottom=0.05, right=0.95, hspace=0.3)
        zipp = zip(
                axes.T,
                np.array(stats).T[[4, 5, 10]],
                means[[4,5,10]],
                [np.sqrt(covs[0, 0]), np.sqrt(covs[1, 1]), 0.0],
                labels,
                colors,
            )
    if not anisotropic: 
        labels = [r"$\sigma_{\alpha}$", r"$\chi^{2}$"]
        colors = ["r", "g"]
        fig, axes = plt.subplots(figsize=(7, 2), nrows=1, ncols=2, sharey=True, squeeze=False)
        plt.subplots_adjust(left=0.1, top=0.95, bottom=0.05, right=0.95, hspace=0.3)
        zipp = zip(
                axes.T,
                np.array(stats).T[[1, 2]],
                means[[1,2]],
                [np.sqrt(covs[0, 0]), 0.0],
                labels,
                colors,
            )
    
    if varybetaphaseshift:
        if anisotropic: 
            labels = [r"$\sigma_{\alpha}$", r"$\sigma_{\alpha_{ap}}$",  r"$\sigma_{\beta_{\phi}}$", r"$\chi^{2}$"]
            colors = ["r", "b", "orange", "g"]
            fig, axes = plt.subplots(figsize=(7, 2), nrows=1, ncols=4, sharey=True, squeeze=False)
            plt.subplots_adjust(left=0.1, top=0.95, bottom=0.05, right=0.95, hspace=0.3)
            zipp = zip(
                axes.T,
                np.array(stats).T[[5, 6, 9, 13]],
                means[[5, 6, 9, 13]],
                [np.sqrt(covs[0, 0]), np.sqrt(covs[1, 1]), np.sqrt(covs[4, 4]), 0.0],
                labels,
                colors,
            )
        if not anisotropic:
            labels = [r"$\sigma_{\alpha}$", r"$\sigma_{\beta_{\phi}}$", r"$\chi^{2}$"]
            colors = ["r", "orange", "g" ]
            fig, axes = plt.subplots(figsize=(7, 2), nrows=1, ncols=3, sharey=True, squeeze=False)
            plt.subplots_adjust(left=0.1, top=0.95, bottom=0.05, right=0.95, hspace=0.3)
            zipp = zip(
                axes.T,
                np.array(stats).T[[2, 3, 5]],
                means[[2, 3, 5]],
                [np.sqrt(covs[0, 0]), np.sqrt(covs[1, 1]), 0.0],
                labels,
                colors,
            )
    
    for i, (ax, vals, avgs, stds, l, c) in enumerate(zipp):

        ax[0].hist(vals, 10, color=c, histtype="stepfilled", alpha=0.2, density=False, zorder=0)
        ax[0].hist(vals, 10, color=c, histtype="step", alpha=1.0, lw=1.3, density=False, zorder=1)
        
        # ax[0].axvline(data_sig[i], color="k", ls="-", zorder=2)
        if l != r"$\chi^{2}$":
            ax[0].axvline(avgs, color="k", ls="--", zorder=2)
            ax[0].axvline(stds, color="k", ls=":", zorder=2)
        ax[0].set_xlabel(l)
        
        axes[0, 0].set_ylabel(r"$N_{\mathrm{mocks}}$")

    plt.show()

    fig.savefig(figname, bbox_inches="tight", transparent=True, dpi=300)


# plot the finished chains 

In [ ]:
# Everything below here is for plotting the chains once they have been run. The should_plot()
# function will check for the presence of chains and plot if it finds them on your laptop. On the HPC you can
# also force this by passing in "plot" as the second argument when calling this code from the command line.
datanames = ['spline', 'poly', 'spline phaseshift', 'poly phaseshift']

import logging

logging.info("Creating plots")
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

for dataname in datanames:
    for recon in ["prerecon", "postrecon"]:
        plotname = f"{dataname}_{recon}"
        dir_name = "/".join(pfn.split("/")[:-1]) + "/" + plotname
        try:
            if not os.path.exists(dir_name):
                os.makedirs(dir_name, exist_ok=True)
        except Exception:
            pass
        
        

# Loop over all the fitters
c = [ChainConsumer() for i in range(2 * len(datanames))]
stats = [[[], []] for _ in range(len(datanames))]

for posterior, weight, chain, evidence, model, data, extra in fitter.load():
    # continue 
    
    datname = extra['name'] 
    if '$\\beta_{\\phi(N_{\\mathrm{eff}})}$' in model.get_labels():
        datname = datname + ' phaseshift'
        
    if 'poly' == model.broadband_type: 
        datname = datname + ' poly'

    else:
        datname = datname + ' spline'

    data_bin = 0
    if model.broadband_type == 'poly':
        data_bin = 1
    if '$\\beta_{\\phi(N_{\\mathrm{eff}})}$' in model.get_labels():
        data_bin += 2
        
    recon_bin = 0 if "Prerecon" in extra["name"] else 1
    stats_bin = recon_bin * len(datanames) + data_bin
    realisation = str(extra["name"].split()[-1]) if "realisation" in extra["name"] else "mean"
    # print(extra["name"], data_bin, recon_bin, stats_bin, realisation)

    # Store the chain in a dictionary with parameter names
    df = pd.DataFrame(chain, columns=model.get_labels())

    if "$\\epsilon$" in model.get_labels(): 
        # Compute alpha_par and alpha_perp for each point in the chain
        alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
        df["$\\alpha_\\parallel$"] = alpha_par
        df["$\\alpha_\\perp$"] = alpha_perp
        df["$\\alpha_{ap}$"] = (1.0 + df["$\\epsilon$"].to_numpy()) ** 3
        newweight = np.where(
            np.logical_and(
                np.logical_and(df["$\\alpha_\\parallel$"] >= 0.6, df["$\\alpha_\\parallel$"] <= 1.4),
                np.logical_and(df["$\\alpha_\\perp$"] >= 0.6, df["$\\alpha_\\perp$"] <= 1.4),
            ),
            weight,
            0.0,
        )
    else:
        newweight = np.where(
            np.logical_and(df["$\\alpha$"] >= 0.8, df["$\\alpha$"] <= 1.2),
            weight,
            0.0,
        )
    # Get the MAP point and set the model up at this point
    model.set_data(data)
    r_s = model.camb.get_data()["r_s"]
    max_post = posterior[newweight > 0].argmax()
    params = df[newweight > 0].iloc[max_post]
    params_dict = model.get_param_dict(chain[newweight > 0][max_post])
    for name, val in params_dict.items():
        model.set_default(name, val)
    

    paramscov = [
                "$\\alpha$",
                "$\\alpha_{ap}$",
                "$\\alpha_\\parallel$",
                "$\\alpha_\\perp$",
            ]
    if "$\\epsilon$" not in model.get_labels(): 
        paramscov = [
                "$\\alpha$",
            ]
    if '$\\beta_{\\phi(N_{\\mathrm{eff}})}$' in model.get_labels():
        paramscov.append(
                '$\\beta_{\\phi(N_{\\mathrm{eff}})}$'
        ) 
        
    # Compute some summary statistics and add them to a dictionary
    mean, cov = weighted_avg_and_cov(
        df[
            paramscov
        ],
        newweight,
        axis=0,
    )

    # Add the chain or MAP to the Chainconsumer plots
    extra.pop("realisation", None)
    #print(extra)
    if realisation == "mean":
        extra.pop("color", None)
        c[stats_bin].add_chain(
            Chain(
            samples=df, name=datname, weights=newweight, 
            color="k", realisation=None, plot_contour=True, 
            plot_point=False, show_as_1d_prior=False
            )
        )
        figname = None
        mean_mean, cov_mean = mean, cov
        
    else:
        #continue 
        # print(params)
        c[stats_bin].add_marker(location=params.to_dict(), marker_size=50, name=datname)
        # Get some useful properties of the fit, and plot the MAP model against the data if the bestfit alpha or alpha_ap are outliers compared to the mean fit
#         if '$\\beta_{\\phi(N_{\\mathrm{eff}})}$' in model.get_labels():
#             if "\\epsilon" in model.get_labels():
#                 diff = np.c_[params["$\\alpha_\\parallel$"], params["$\\alpha_\\perp$"], params['$\\beta_{\\phi(N_{\\mathrm{eff}})}$']] - mean_mean[2:]
#             else:
#                 diff = np.c_[params["$\\alpha$"], params['$\\beta_{\\phi(N_{\\mathrm{eff}})}$']] - mean_mean
                
#         else:
#             if "$\\epsilon$" in model.get_labels():
#                 diff = np.c_[params["$\\alpha_\\parallel$"], params["$\\alpha_\\perp$"]] - mean_mean[2:]
#             else:
#                 diff = np.c_[params["$\\alpha$"]] - mean_mean
                
#         outlier = diff @ np.linalg.inv(cov_mean[2:, 2:]) @ diff.T
        # if outlier > sp.stats.chi2.ppf(0.9545, 2, loc=0, scale=1):
        # dataname = extra["name"].split(" ")[3].lower()
        # plotname = f"{datanames[data_bin]}_prerecon" if recon_bin == 0 else f"{datanames[data_bin]}_postrecon"
        # figname = "/".join(pfn.split("/")[:-1]) + "/" + plotname + "/" + extra["name"].replace(" ", "_") + "_contour.png"
        # # continue 
        # if not os.path.isfile(figname):
        #     extra.pop("color", None)
        #     cc = ChainConsumer()
        #     cc.add_chain(Chain(samples=df, name=datname, weights=newweight, color=colors[data_bin]))
        #     cc.add_marker(location=df.iloc[max_post].to_dict(), **extra)
        #     cc.plotter.plot(filename=figname)
        #     figname = "/".join(pfn.split("/")[:-1]) + "/" + plotname + "/" + extra["name"].replace(" ", "_") + "_bestfit.png"
        # else:
        #     figname = None

    new_chi_squared, dof, bband, mods, smooths = model.simple_plot(
        params_dict, display=False, figname=figname, title=datname, c=colors[data_bin]
    )
    if realisation == "mean":
        print(25.0 * new_chi_squared, dof)

#     if data_bin == 0 and (realisation == "2" or realisation == "21" or realisation == "22"):
#         df["weight"] = newweight
#         df.to_csv("/".join(pfn.split("/")[:-1]) + "/" + plotname + f"_BOSSpoly.dat", index=False, sep=" ")

    if '$\\beta_{\\phi(N_{\\mathrm{eff}})}$' in model.get_labels(): 
        
        if "$\\epsilon$" in model.get_labels(): 
            stats[data_bin][recon_bin].append(
                [
                    mean[0], # mean alpha 
                    mean[1], # mean alpha aps 
                    mean[2], # mean alpha para 
                    mean[3], # mean alpha perp 
                    mean[4], # beta phase shift 
                    np.sqrt(cov[0, 0]), 
                    np.sqrt(cov[1, 1]),
                    np.sqrt(cov[2, 2]),
                    np.sqrt(cov[3, 3]),
                    np.sqrt(cov[4, 4]),
                    cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1]), # cov alpha/alpha_AP
                    cov[2, 3] / np.sqrt(cov[2, 2] * cov[3, 3]), # cov alpha para/perp 
                    cov[0, 4] / np.sqrt(cov[0, 0] * cov[4, 4]), # cov alpha/beta phaseshift 
                    new_chi_squared,
                    params_dict["alpha"],
                    params_dict["epsilon"],
                    params_dict["beta_phase_shift"],
                ]
            )
        else:
            stats[data_bin][recon_bin].append(
                [
                    mean[0], # mean alpha 
                    mean[1], # beta phase shift 
                    np.sqrt(cov[0, 0]), 
                    np.sqrt(cov[1, 1]),
                    cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1]), # cov alpha/ beta phase shift 
                    new_chi_squared,
                    params_dict["alpha"],
                    params_dict["beta_phase_shift"],
                ]
            )
    
    else:
        
        if "$\\epsilon$" in model.get_labels(): 
        
            stats[data_bin][recon_bin].append(
                [
                    mean[0], # mean alpha 
                    mean[1], # mean alpha aps 
                    mean[2], # mean alpha para 
                    mean[3], # mean alpha perp 
                    np.sqrt(cov[0, 0]), 
                    np.sqrt(cov[1, 1]),
                    np.sqrt(cov[2, 2]),
                    np.sqrt(cov[3, 3]),
                    cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1]), # cov alpha/alpha_AP
                    cov[2, 3] / np.sqrt(cov[2, 2] * cov[3, 3]), # cov alpha para/perp 
                    new_chi_squared,
                    params_dict["alpha"],
                    params_dict["epsilon"],
                ]
            ) 
        else: 
            stats[data_bin][recon_bin].append(
                [
                    mean[0], # mean alpha 
                    np.sqrt(cov), 
                    new_chi_squared,
                    params_dict["alpha"],
                ]
            ) 
print('done')        

# write the best fits to a file 

In [ ]:
loc = '/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/secondgenv1_2_'

for i, name in enumerate(datanames):
    if i < 2:
        continue 
    for j, recon in enumerate(['pre', 'post']):
        
        #print(name, recon)
        dat = stats[i][j]
        #print(dat)
        if mocks[:-3] in ['ELGs_z08_11', 'QSOs_z08_21', 'BGS_z01_04']: # no epsilon 
            
            if i > 1: 
                
                colnames = ['<alpha>', '<beta_phi>', 'sigma_alpha', 'sigma_beta_phi', 'rho_alpha-beta_phi', 'chisquare', 
                            'max posterior alpha', 'max posterior beta_phi']
  
            else:
                
                colnames = ['<alpha>', 'sigma_alpha', 'chisquare', 'max posterior alpha']
                
                
        else: # there is epsilon
            
            if i > 1: 
                  
                colnames = ['<alpha>', '<alpha_AP>', '<alpha_para>', '<alpha_perp>', '<beta_phi>', 
                            'sigma_alpha', 'sigma_alpha_AP', 'sigma_alpha_para', 'sigma_alpha_perp', 'sigma_beta_phi', 
                            'rho_alpha-alpha_AP', 'rho_alpha_para-alpha_perp', 'rho_alpha-beta_phi', 
                            'chisquare', 
                            'max posterior alpha', 'max posterior epsilon', 'max posterior beta_phi']
            else:
                
                colnames = ['<alpha>', '<alpha_AP>', '<alpha_para>', '<alpha_perp>',
                            'sigma_alpha', 'sigma_alpha_AP', 'sigma_alpha_para', 'sigma_alpha_perp',
                            'rho_alpha-alpha_AP', 'rho_alpha_para-alpha_perp',
                            'chisquare', 
                            'max posterior alpha', 'max posterior epsilon']
                
        dat = pd.DataFrame(dat)
        dat.columns = colnames
        #print(dat)
        
        dat['mock'] = ['mean', '0', '1', '2', '3', '4', '5', '6','7', '8', '9', '10', '11', '12', '13', '14', '15', '16','17', '18', 
                      '19', '20', '21', '22', '23', '24']

        dat.to_csv(loc + name + '_' + recon + '_' + mocks + '.csv', index=False)

        

# plot mean of the mocks fits + markers for other fits 

In [ ]:

for j in np.arange(2,4):#range(4):
    for k in range(2):
        stats_bin = k*4 + j 
        
        # if j > 1:
        #     continue 
        rec = ''
        if k == 0:
            rec = 'prerecon'
        else:
            rec = 'postrecon'
            
        print('mock mean' + mocks + datanames[j] + rec)

        c[stats_bin].set_plot_config(PlotConfig(show_legend=False))
        cols = ['$\\alpha$','$\\alpha_{ap}$']
        if mocks[:-3] in ['BGS_z01_04', 'ELGs_z08_11', 'QSOs_z08_21']: 
            cols = ['$\\alpha$']
        if j > 1: 
            cols = ['$\\alpha$','$\\alpha_{ap}$', '$\\beta_{\\phi(N_{\\mathrm{eff}})}$']
            if mocks[:-3] in ['BGS_z01_04', 'ELGs_z08_11', 'QSOs_z08_21']: 
                cols = ['$\\alpha$', '$\\beta_{\\phi(N_{\\mathrm{eff}})}$']
        c[stats_bin].plotter.plot(columns=cols, 
        filename="/".join(pfn.split("/")[:-1]) + "/" +'mock mean' + mocks + datanames[j] + rec,
        )
        if j <= 1:
            print('-------------------------------------')
            print(c[stats_bin].analysis.get_latex_table(columns=cols))
            print('-------------------------------------')
            
        else:
            print('-------------------------------------')
            print(c[stats_bin].analysis.get_latex_table(columns=cols))
            print('-------------------------------------')
            

# plot errors on fits histograms 

In [ ]:

for recon_bin in range(2):
    for data_bin in range(4):
        # continue 
        dataname = datanames[data_bin]
        stats_bin = recon_bin * len(datanames) + data_bin
        i = tracers[t].index(z)

        mean = np.mean(stats[data_bin][recon_bin][1:], axis=0)
        cov = np.cov(stats[data_bin][recon_bin][1:], rowvar=False)

#         c[stats_bin].from_covariance(
#             mean[:4],
#             cov[:4, :4],
#             parameters=["$\\alpha$", "$\\alpha_{ap}$", "$\\alpha_\\parallel$", "$\\alpha_\\perp$"],
#             color=colors[data_bin],
#             plot_contour=True,
#             plot_point=False,
#             show_as_1d_prior=False,
#         )

#         truth = {
#             "$\\alpha$": 1.0,
#             "$\\alpha_{ap}$": 1.0,
#             "$\\alpha_\\perp$": 1.0,
#             "$\\alpha_\\parallel$": 1.0,
#             "$\\Sigma_{nl,||}$": sigma_nl_par[t][i][recon_bin],
#             "$\\Sigma_{nl,\\perp}$": sigma_nl_perp[t][i][recon_bin],
#             "$\\Sigma_s$": sigma_s[t][i][recon_bin],
#         }

#         plotname = f"{dataname}_prerecon" if recon_bin == 0 else f"{dataname}_postrecon"
#         c[stats_bin].plotter.plot(
#             filename=["/".join(pfn.split("/")[:-1]) + "/" + plotname + f"_contour.png"],
#             truth=truth,
#             parameters=[
#                 "$\\alpha_\\parallel$",
#                 "$\\alpha_\\perp$",
#             ],
#             legend=False,
#         )
#         c[stats_bin].plotter.plot(
#             filename=["/".join(pfn.split("/")[:-1]) + "/" + plotname + f"_contour2.png"],
#             truth=truth,
#             parameters=[
#                 "$\\alpha$",
#                 "$\\alpha_{ap}$",
#             ],
#             legend=False,
#         )

        # Plot histograms of the chi squared values and uncertainties for comparison to the data

        # data_sig = data_sigmas_prerecon[t][i] if recon_bin == 0 else data_sigmas_postrecon[t][i]
        
        plotname = f"{dataname}_prerecon" if recon_bin == 0 else f"{dataname}_postrecon"
        plot_errors(stats[data_bin][recon_bin], "/".join(pfn.split("/")[:-1]) + "/" + plotname + f"_errors.png",
            varybetaphaseshift=True if data_bin > 1 else False, 
            anisotropic=False if mocks[:-3] in ['BGS_z01_04', 'ELGs_z08_11', 'QSOs_z08_21'] else True)
        np.save("/".join(pfn.split("/")[:-1]) + "/Summary_" + plotname + f".npy", stats[data_bin][recon_bin])

# plotting combinations of the mocks 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer, Chain, Truth, PlotConfig     

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


path_p1 = '/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/'

dat = '_pk' 

pfns = []
# listnames = ['BGS_z01_04', 'LRGs_z04_06', 'LRGs_z06_08', 'ELGsLRGscombined_z08_11', 'ELGs_z11_16', 'QSOs_z08_21', 'LRGs_z08_11', 'ELGs_z08_11']#,
listnames = ['BGS_z01_04', 'LRGs_z04_06', 'LRGs_z06_08', 'ELGs_z11_16', 'QSOs_z08_21', 'LRGs_z08_11', 'ELGs_z08_11']#,

for i in listnames:
    pfns.append(path_p1 + 'desi_kp4_SecondGen_' + i + dat + '/output/desi_kp4_SecondGen_' + i + dat + '.fitter.pkl')


c = ChainConsumer() 

for i,p in enumerate(pfns): 
    #print(i)
    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    count = 0
    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        
        if 'mean' not in extra['name']:  # making sure we only get chain for mock mean 
            continue 
            
        if 'Prerecon' not in extra['name']: # making sure we only get results from recon 
            continue  
                
        if '$\\beta_{\\phi(N_{\\mathrm{eff}})}$' not in model.get_labels():
                continue 
                
        df = pd.DataFrame(chain, columns=model.get_labels())
    
        # Compute alpha_par and alpha_perp for each point in the chain
#         alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
#         df["$\\alpha_\\parallel$"] = alpha_par
#         df["$\\alpha_\\perp$"] = alpha_perp
        
        # Add the chain or MAP to the Chainconsumer plots
        #print(extra)
        extraname = ''
        if model.broadband_type == 'spline':
            extraname = ' spline' 
        else:
            extraname = 'poly'
            
        extra.pop("realisation", 'mean' + extraname)
        # extra["name"] = names[i] + extraname
        extra["name"] = extra['name'] + extraname
        
       
        c.add_chain(Chain(samples=df, weights=weight, name=listnames[i] + extraname, plot_contour=True, plot_point=False, show_as_1d_prior=False))
        
        # model.set_data(data)
        # r_s = model.camb.get_data()["r_s"]
        # max_post = posterior.argmax()
        # params = df.loc[max_post]
        # params_dict = model.get_param_dict(chain[max_post])
        # for name, val in params_dict.items():
        #     model.set_default(name, val)

        # new_chi_squared, dof, bband, mods, smooths = model.plot(params_dict, display=False)
        # k = model.data[0]["ks"]
        # axes[i][count].scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # axes[i][count].plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # axes[i][count].legend()
        
        #plt.scatter(k, (model.data[0]['pk0'][0]- smooths[0][0])*k, label= extraname + ' data')
        # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.legend()
        # count += 1

        
        # if i == 0:
        #     default = smooths[0][0]
        # # plt.plot(k, (mods[0][0] - smooths[0][0])*k, label=extraname)
        # plt.plot(k, (smooths[0][0]/default)*k, label=extraname)
        
        
    
# alpha_para, alpha_perp, beta = 0.97912566, 0.98881584, 1.0

alpha_para, alpha_perp = 1.0, 1.0 # 1.00093140, 1.01083738
#couplingterm = 1.0/0.22710731766023898
#beta = (3.044/(3.044+couplingterm)) / (3.7/(3.7+couplingterm))  
beta = 1.0

alpha, eps = 1.0, 0.0 #model.get_reverse_alphas(alpha_para, alpha_perp)
   
truth = {"$\\alpha$": alpha, "$\\epsilon$": eps, 
        #  "$\\alpha_\\parallel$": alpha_para, "$\\alpha_\\perp$": alpha_perp, 
         '$\\beta_{\\phi(N_{\\mathrm{eff}})}$': 1.0
        }

parameters = [names for names in truth]

# plt.legend()
# plt.xlabel(r'$k$ $\mathrm{Mpc}^{-1} h$')
# plt.ylabel(r'$k P_0(k)$ $\mathrm{Mpc}^2 h^{-2}$')
c.set_plot_config(PlotConfig(label_font_size=10))
c.add_truth(Truth(location=truth)) 
c.plotter.plot(columns=['$\\alpha$', '$\\epsilon$', '$\\beta_{\\phi(N_{\\mathrm{eff}})}$'])
#     truth=truth,
#         parameters = parameters[:2]+parameters[4:], 
#         legend=True,
#         display=False,
#         figsize=(13, 13), 
# )